In [1]:
# 安裝 OpenAI SDK 與 diffusers、transformers、accelerate 以及 Pillow
!pip install --quiet openai==0.28 diffusers transformers accelerate scipy safetensors pillow

# 驗證 GPU 是否可用
import torch
print("CUDA available:", torch.cuda.is_available())


CUDA available: True


In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')

In [3]:
import openai

# 從環境變數讀取 API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# ========== Step 1. 設定「主題」 ========== #
theme = "考試作弊"  # ← 如果要生成其他題材的四格笑話，就改這裡

# ========== Step 2. 設計 Prompt 並呼叫 GPT-4 ========== #
prompt = f"""
你是一位四格漫畫腳本撰寫專家，請用幽默、簡潔的口吻，根據主題「{theme}」，寫出更有節奏的四格腳本，注意：
- 第一格：角色要一開始就顯得很慌張。
- 第二格：強調他如何偷偷偷答案。
- 第三格：加入監考老師出現的搞笑誤會。
- 第四格：要有爆點反轉，讓讀者意想不到。
每格只要 1～2 句對白即可。
"""

response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7,
    max_tokens=300
)

# 取得 GPT-4 回傳的文字腳本
script = response.choices[0].message.content.strip()
print("=== 最終文字腳本 ===")
print(script)


=== 最終文字腳本 ===
**第一格：**  
（角色緊張地看著考卷）  
角色：天啊，這題我根本不會！

**第二格：**  
（角色偷偷撇頭，手上拿著小紙條）  
角色：快點，看看我的作弊小抄！

**第三格：**  
（監考老師突然出現，誤會角色在寫信）  
監考老師：小子，考試期間寫情書可不行！

**第四格：**  
（角色慌忙把小紙條塞進嘴裡）  
角色：這可不是情書，這是我的「解題秘笈」！


In [4]:
# 把 script 存成本地的 .txt 檔，方便下載與交付
with open("comic_script.txt", "w", encoding="utf-8") as f:
    f.write(script)

print("已將文字腳本存檔為 comic_script.txt")
# 如果想要在 Colab 左側「Files」面板看到，請執行下一行：
!ls -l comic_script.txt


已將文字腳本存檔為 comic_script.txt
-rw-r--r-- 1 root root 478 Jun  5 08:36 comic_script.txt


In [5]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

# ========== Step 1. 載入模型到 GPU ========== #
model_id = "stabilityai/stable-diffusion-xl-base-1.0"  # 若要改用 v1-5，改成 "runwayml/stable-diffusion-v1-5"

# 使用 fp16 加速與省記憶體
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

# 關閉安全檢查（可選）
pipe.safety_checker = None

# ========== Step 2. 解析文字腳本、拆分成四段 Prompt ========== #

# 假設 script 格式如下（範例）：
# 1. 【第一格】：大雄坐在考場，心臟狂跳，滿頭大汗。心裡話：「好緊張，絕對不能失手……」
# 2. 【第二格】：他偷偷把手伸到隔壁小叮噹的桌下，想拿答案小紙條。對白：「只要偷到就好……」
# 3. 【第三格】：監考老師誤以為大雄在禮貌舉手，馬上滑步過來。老師對白：「有問題嗎？」
# 4. 【第四格】：大雄驚恐舉手，結果手上拿的是作弊紙條，老師一臉懵。爆點對白：「我……我只是想提醒自己……『不要作弊』！」

# 為了方便示範，我們不使用程式自動解析，而是直接把四段文字填到下面 prompt_frame1 ～ prompt_frame4
prompt_frame1 = (
    "手繪四格漫畫風格，線條分明，色彩鮮明，"
    "角色：大雄坐在考場，心臟狂跳，滿頭大汗，漫畫場景顯示其他同學看試卷，表情非常焦慮。"
)
prompt_frame2 = (
    "手繪四格漫畫風格，線條分明，色彩鮮明，"
    "角色：大雄偷偷把手伸到隔壁小叮噹桌下，手上露出作弊紙條，小叮噹露出疑惑表情。"
)
prompt_frame3 = (
    "手繪四格漫畫風格，線條分明，色彩鮮明，"
    "角色：監考老師滑步走向大雄，以為他在禮貌舉手，老師表情認真，背景為考場長凳與黑板。"
)
prompt_frame4 = (
    "手繪四格漫畫風格，線條分明，色彩鮮明，"
    "角色：大雄驚恐地舉起作弊紙條，老師一臉懵懂，畫面要有爆點感，誇張漫畫表情。"
)

# ========== Step 3. 一格一格生成並存檔 ========== #

# 這個 helper function 可以重複使用
def generate_and_save(prompt: str, filename: str, width: int = 512, height: int = 512, steps: int = 30, scale: float = 7.5):
    """
    - prompt: 要輸入給 SD 的提示文字
    - filename: 儲存的檔名 (如 "frame1.png")
    - width, height: 影像解析度 (SDXL 預設可到 1024，但 CPU 上建議 512)
    - steps: 推論步數 (30 ～ 50 之間)
    - scale: guidance scale (7.5～8.5 足以)
    """
    image = pipe(
        prompt,
        width=width,
        height=height,
        num_inference_steps=steps,
        guidance_scale=scale
    ).images[0]
    image.save(filename)
    print(f"已儲存：{filename}")

# 利用 GPU，我們可以用 512×512、steps=30
generate_and_save(prompt_frame1, "frame1.png", width=512, height=512, steps=30, scale=7.5)
generate_and_save(prompt_frame2, "frame2.png", width=512, height=512, steps=30, scale=7.5)
generate_and_save(prompt_frame3, "frame3.png", width=512, height=512, steps=30, scale=7.5)
generate_and_save(prompt_frame4, "frame4.png", width=512, height=512, steps=30, scale=7.5)

# 檢查檔案是否存在
!ls -l frame1.png frame2.png frame3.png frame4.png


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (165 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['�� ， 心 臟 狂 跳 ， 滿 頭 大 汗 ， 漫 畫 場 景 顯 示 其 他 同 學 看 試 卷 ， 表 情 非 常 焦 慮 。']
Token indices sequence length is longer than the specified maximum sequence length for this model (165 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['�� ， 心 臟 狂 跳 ， 滿 頭 大 汗 ， 漫 畫 場 景 顯 示 其 他 同 學 看 試 卷 ， 表 情 非 常 焦 慮 。']


  0%|          | 0/30 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['手 伸 到 隔 壁 小 叮 噹 桌 下 ， 手 上 露 出 作 弊 紙 條 ， 小 叮 噹 露 出 疑 惑 表 情 。']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['手 伸 到 隔 壁 小 叮 噹 桌 下 ， 手 上 露 出 作 弊 紙 條 ， 小 叮 噹 露 出 疑 惑 表 情 。']


...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['�� 起 作 弊 紙 條 ， 老 師 一 臉 懵 懂 ， 畫 面 要 有 爆 點 感 ， 誇 張 漫 畫 表 情 。']


已儲存：frame3.png


The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['�� 起 作 弊 紙 條 ， 老 師 一 臉 懵 懂 ， 畫 面 要 有 爆 點 感 ， 誇 張 漫 畫 表 情 。']


  0%|          | 0/30 [00:00<?, ?it/s]

已儲存：frame4.png
-rw-r--r-- 1 root root 296227 Jun  5 08:37 frame1.png
-rw-r--r-- 1 root root 295895 Jun  5 08:37 frame2.png
-rw-r--r-- 1 root root 282826 Jun  5 08:38 frame3.png
-rw-r--r-- 1 root root 327479 Jun  5 08:38 frame4.png


In [6]:
from PIL import Image

# 讀取剛剛生成的四張 frameX.png
img1 = Image.open("frame1.png")
img2 = Image.open("frame2.png")
img3 = Image.open("frame3.png")
img4 = Image.open("frame4.png")

# 確認每張圖尺寸 (預設都是 512×512)
w, h = img1.size
print("每張圖尺寸：", w, "×", h)

# 建立一張新的空白畫布，大小為 (2w × 2h)
canvas = Image.new("RGB", (w * 2, h * 2), (255, 255, 255))

# 貼上四張圖：左上 (0,0) → 右上 (w,0) → 左下 (0,h) → 右下 (w,h)
canvas.paste(img1, (0, 0))
canvas.paste(img2, (w, 0))
canvas.paste(img3, (0, h))
canvas.paste(img4, (w, h))

# 儲存最終拼貼結果
canvas.save("four_panel_comic.png")
print("已儲存最終四格漫畫：four_panel_comic.png")

# 確認檔案
!ls -l four_panel_comic.png


每張圖尺寸： 512 × 512
已儲存最終四格漫畫：four_panel_comic.png
-rw-r--r-- 1 root root 1210256 Jun  5 08:38 four_panel_comic.png
